In [ ]:
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import glob, os
import numpy as np
import seaborn as sn

In [ ]:
## Function to remove peeks
def remove_peeks(dlf: pd.DataFrame, feature: str,window):
    df= dlf.copy()
    new_Feature_name= 'smoothed_' + feature
    #df[new_Feature_name]= savgol_filter(x=df['T_Spindel'],window_length=33,polyorder=4)
    df[new_Feature_name]= df[feature].rolling(window).median()
    shape= df.shape
    #print(df[df[new_Feature_name] != df[new_Feature_name]].shape[0])
    offset= int(np.round(window/2))
    for i in range(df.shape[0] - offset):
        #print('before: ', df.iloc[i][new_Feature_name])
        df.loc[i, new_Feature_name]= df.iloc[i + offset][new_Feature_name]
       # print('After: ', df.iloc[i][new_Feature_name])
    df= df[df[new_Feature_name] == df[new_Feature_name]]
    #print ('last Result: ', df[new_Feature_name])
    return df

In [ ]:
#Apply shift and cut to the signal
def shift_cut(df:pd.DataFrame):
    Z_TCP_OK = 0
    Z_Schlitten_OK = 0
    Z_Hals_OK = 0
    Z_Welle_OK = 0
    df['z_tcp_ok_orginal']= df['z_tcp_ok'].copy()
    for index, row in df.iterrows():
        if row["drhz"] >=3000:
            ## Apply cut
            df= df[index:]
            ## Apply Shift
            df['z_tcp_ok']= (df['z_tcp_ok'] - Z_TCP_OK)
            df['z_schlitten_ok']= df['z_schlitten_ok'] - Z_Schlitten_OK
            df['z_hals_ok']= df['z_hals_ok'] - Z_Hals_OK
            df['z_welle_ok']= df['z_welle_ok'] - Z_Welle_OK
            
            # df.loc[:,'Z_TCP_OK']= df.loc[:,'Z_TCP_OK'] - Z_TCP_OK
            # df.loc[:,'Z_Schlitten_OK']= df.loc[:,'Z_Schlitten_OK'] - Z_Schlitten_OK
            # df.loc[:,'Z_Hals_OK']= df.loc[:,'Z_Hals_OK'] - Z_Hals_OK
            # df.loc[:,'Z_Welle_OK']= df.loc[:,'Z_Welle_OK'] - Z_Welle_OK
            break
        else:
            Z_TCP_OK= row['z_tcp_ok']
            Z_Schlitten_OK = row['z_schlitten_ok']
            Z_Hals_OK = row['z_hals_ok']
            Z_Welle_OK = row['z_welle_ok']
    return df

In [ ]:
# source_dir= "G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/repaired_Files/Data_Verarbeitet/orginal/"
# columns= ['T_Y_ACHSE','T_Z_ACHSE','DRHZ','Z_SCHLITTEN_MK','Z_HALS_MK','Z_SPINDEL_MK','KOMP','DAT_UHZ', 'T_Bett','T_Spindel',	'Z_TCP_OK','Z_Schlitten_OK','Z_Hals_OK','Z_Welle_OK','Z_TCP_MK', 'M8', 'M21','M121']
# new_data_dir= 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/csvs/smoothed/'
# target_dir="G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/repaired_Files/Data_Verarbeitet/peak_removed_shifted/median/"
source_dir= "G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/repaired_Files/Data_Verarbeitet/orginal/"
#columns= ['T_Y_ACHSE','T_Z_ACHSE','DRHZ','Z_SCHLITTEN_MK','Z_HALS_MK','Z_SPINDEL_MK','KOMP','DAT_UHZ', 'T_Bett','T_Spindel',	'Z_TCP_OK','Z_Schlitten_OK','Z_Hals_OK','Z_Welle_OK','Z_TCP_MK', 'M8', 'M21','M121']
new_data_dir= 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/row_files/combines/'
target_dir="G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/row_files/combines/smoothed/With_cooling_feature/"

In [ ]:
working_dir= new_data_dir
os.chdir(working_dir)
for file in glob.glob('*.CSV'):
    print (file)
    file_dir= working_dir+file
    df= pd.read_csv(file_dir, sep=';', decimal= ',')
    #df= df[columns]
    df.columns= df.columns.str.lower()
    print (df.columns)
    df= remove_peeks(df, feature='t_spindel', window= 16)
    df= shift_cut(df)
    fig, ax= plt.subplots(figsize=(15,15))
    plt.title(file)
    plt.grid()
    #plt.plot(df['Z_TCP_OK'], label= 'Z_TCP_OK_Shifted')
    plt.plot(df['t_spindel'], label= 'orignal')
    plt.plot(df['smoothed_t_spindel'], label= 'smoothed')
    plt.legend()
    plt.show()
    file_name= str('smoothed_'+file)
    df.to_csv(str(working_dir+'smoothed/'+file_name))

In [ ]:
##Combine created files
working_dir= target_dir#new_data_dir + 'all_files/'# target_dir
os.chdir(working_dir)
# columns= ['T_Y_ACHSE','T_Z_ACHSE','DRHZ','Z_SCHLITTEN_MK','Z_HALS_MK','Z_SPINDEL_MK','KOMP','DAT_UHZ', 'T_Bett','smoothed_T_Spindel', 'T_Spindel',	'Z_TCP_OK','Z_Schlitten_OK','Z_Hals_OK','Z_Welle_OK','Z_TCP_MK', 'M8', 'M21','M121','Z_TCP_OK_Orginal']
#result= pd.DataFrame(columns= columns)
# result.columns= result.columns.str.lower()
# columns= result.columns
# result= pd.DataFrame()
#selected_Columns= ['t_z_achse','t_y_achse','drhz', 't_bett','t_spindel','smoothed_t_spindel', 't_motor','komp', 'z_tcp_ok', 'z_tcp_ok_orginal']
selected_Columns= ['t_spindel', 't_motor', 't_kuehlung','t_kss_tank', 't_raum',
      't_spindel_rueck', 't_spindel_vor', 'volumenstrom', 'M8', 'M121',
      'M127', 'M7', 'z_welle_ok']
result= None#pd.DataFrame(columns= selected_Columns)
for file in glob.glob('*.csv'):
    if file == 'combined.csv':
        continue
    df= pd.read_csv(file)
    if result is  None:
        result= df
    else:
        result= pd.concat([result, df])
    print ('file name: ', file, ' nans: ', result.isna().sum().sum())
result.to_csv((working_dir+'combined_file/'+'combined.csv'))

In [ ]:
## Load combined File
#df= pd.read_csv(str(new_data_dir+'all_files/combined.csv'))
df= pd.read_csv(str(working_dir+'/combined_file/'+'combined.csv'))
#selected_Columns= ['t_y_achse','drhz', 't_bett','t_spindel','smoothed_t_spindel', 'm8', 'm21','m121', 'z_tcp_ok', 'z_tcp_ok_orginal']
#selected_Columns= ['t_z_achse','t_y_achse','drhz', 't_bett','t_spindel','smoothed_t_spindel', 't_motor', 'z_tcp_ok', 'komp', 'z_tcp_ok_orginal']
#selected_Columns= [ 't_spindel', 'drhz', 't_motor', 't_spindel_vor', 'z_tcp_ok']

selected_Columns= [  'dat_uhz',
       't_bett', 't_spindel', 't_y_achse', 't_z_achse', 'drhz',
        'z_schlitten_ok',  'z_hals_ok',
        'z_spindel_ok', 'komp', 't_motor',
       'z_welle_ok', 't_kss_tank', 't_raum',
       't_spindel_rueck', 't_spindel_vor', 'volumenstrom',
       'smoothed_t_spindel', 'z_tcp_ok_orginal', 'z_tcp_ok']

selected_Columns= ['t_spindel', 't_motor', 't_kuehlung','t_kss_tank', 't_raum',
      't_spindel_rueck', 't_spindel_vor', 'volumenstrom', 'M8', 'M121',
      'M127', 'M7', 'z_welle_ok']
selected_Columns= ['t_spindel', 't_motor', 't_kuehlung', 'z_welle_ok']
print(df.columns)
x= df[selected_Columns]

#x['dif']= x['t_motor'] - x['t_spindel']
#y= df['z_tcp_ok']

In [ ]:
## create correlation matrix
correlations= x.corr()
fig, ax= plt.subplots(figsize=(15,15))
sn.heatmap(correlations, annot=True, vmin=-1, vmax=1, annot_kws={"size": 16, 'color': 'black'})
plt.show()

In [ ]:
#working_dir= new_data_dir+'all_files/'
working_dir= working_dir= 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/row_files/for_combination/smoothed/'#target_dir
os.chdir(working_dir)
for file in glob.glob('*.CSV'):
    if file != 'combined.csv':
        df= pd.read_csv(file)
        print(file)
        e= str(file).find('BBS_N_')
        print (e)
        if str(file).find('_BBS_N_')!=-1:
            df['t_kuehlung']= df['t_bett']
            print('1')
        elif str(file).find('_BBS_FW_20')!=-1:
            df['t_kuehlung']= 20
            print('2')
        elif str(file).find('_BBS_FW_25')!=-1:
            df['t_kuehlung']= 25
            print('3')
        elif str(file).find('_BBS_FW_30')!=-1:
            df['t_kuehlung']= 30
            print('4')

        print(df['t_kuehlung'].head(3))
        print('##############')
        df.to_csv(str(working_dir+'With_cooling_feature/'+file))